In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 89% 23.0M/25.7M [00:01<00:00, 21.0MB/s]
100% 25.7M/25.7M [00:01<00:00, 24.1MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df_orig = pd.read_csv('IMDB Dataset.csv')
df_orig.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
 df_orig.shape


(50000, 2)

In [137]:
# Data Cleaning part
# Randomly selecting some reviews set of 15000
df= df_orig
df = df.sample(12000)
df.shape


(35000, 2)

In [8]:
# gathering infoabout the data 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 44788 to 10024
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     1200 non-null   object
 1   sentiment  1200 non-null   object
dtypes: object(2)
memory usage: 28.1+ KB


In [9]:
# Checking the count of null values
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [142]:
# Replacing sentiment value column in positive and negative as 0 and 1
from  sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['sentiment']= encoder.fit_transform(df['sentiment'])
df.head()
# this has encoded negative as 0 positive as 1

,review,sentiment
10948,"""Perhaps we can arrange a meet. "" ""Where are y...",1
39501,How does this movie suck? As a fan of Michael ...,0
27055,"I think every critic who panned ""Tommy Boy"" wh...",1
12348,Dramatic ? Yes......Historically accurate ? No...,0
27516,"If you've seen Atlantis 1, then you'd know tha...",0


In [ ]:
# Replacing sentiment value column in positive and negative with and 0 resectively
# df['sentiment'].replace({'positive':1, 'negative':0}), inplace = True)

In [143]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [144]:
# Preprocessor function
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

def preprocess_text(text):
    # remove HTML tags
    text = re.sub('<[^>]*>', '', text)
    
    # lowercase all characters
    text = text.lower()
    
    # remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    text_tokens = nltk.word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    text = ' '.join(filtered_text)
    
    # perform stemming
    porter = PorterStemmer()
    text_tokens = nltk.word_tokenize(text)
    stemmed_text = [porter.stem(word) for word in text_tokens]
    text = ' '.join(stemmed_text)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
df['review'] = df['review'].apply(preprocess_text)

In [14]:
df['review']

44788    whenev hear movi tout sex violenc bad languag ...
43162    perhap time declar intern year cinemat crocodi...
742      mario lewi competit enterpris institut written...
31407    much way spoiler sinc plot reveal still guess ...
21684    day rare come kid show involv preteen teen pop...
                               ...                        
41465    start finish classic two reeler hal roach stud...
26301    film come along manag absolut terribl open tit...
7480     intrud dust dir clarenc brown product mgmexcel...
12584    movi age well mayb impact art character act di...
10024    pain pain word describ aw rendit fun interest ...
Name: review, Length: 1200, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X=cv.fit_transform(df['review']).toarray()
X.shape

(1200, 13851)

In [16]:
y=df['sentiment'].values
y.shape

(1200,)

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [30]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [31]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [32]:
gnb.fit(X_train,y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

0.6375
[[84 38]
 [49 69]]
0.6448598130841121


In [33]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

0.8291666666666667
[[102  20]
 [ 21  97]]
0.8290598290598291


In [34]:
bnb.fit(X_train,y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

0.8333333333333334
[[103  19]
 [ 21  97]]
0.8362068965517241


In [115]:
# checking with LR
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()


**Checking other algorithams also, to see how they perorm**
---



In [116]:
y_pred4 = lr.predict(X_test)
print(accuracy_score(y_test, y_pred4))
print(confusion_matrix(y_test, y_pred4))
print(precision_score(y_test, y_pred4))

0.8333333333333334
[[100  22]
 [ 18 100]]
0.819672131147541


In [117]:
from sklearn.metrics import mean_squared_error
# Calculate training error
y_pred_train = lr.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train)
print('Training MSE: ', train_mse)
# Calculate validation error
val_mse = mean_squared_error(y_test, y_pred4)
print('Validation MSE: ', val_mse)


Training MSE:  0.0
Validation MSE:  0.16666666666666666


In [118]:
# Checking with SVM
from sklearn.svm import SVC
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [119]:
sv = SVC()
sv.fit(X_train,y_train)
sv_pred1 =sv.predict(X_test)
print(accuracy_score(y_test,sv_pred1))
print(confusion_matrix(y_test,sv_pred1))
print(precision_score(y_test,sv_pred1))


0.7916666666666666
[[ 85  37]
 [ 13 105]]
0.7394366197183099


In [120]:
# Calculate training error
y_pred_train_sv = sv.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train_sv)
print('Training MSE: ', train_mse)
# Calculate validation error
val_mse = mean_squared_error(y_test, sv_pred1)
print('Validation MSE: ', val_mse)

Training MSE:  0.028125
Validation MSE:  0.20833333333333334


In [121]:
# checking with Decision Tree
from sklearn.tree import DecisionTreeClassifier
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
DTC = DecisionTreeClassifier()

In [122]:
DTC.fit(X_train,y_train)
DTC_pred =DTC.predict(X_test)
print(accuracy_score(y_test,DTC_pred))
print(confusion_matrix(y_test,DTC_pred))
print(precision_score(y_test,DTC_pred))

0.725
[[85 33]
 [33 89]]
0.7295081967213115


In [123]:
# Calculate training error
y_pred_train_dtc = DTC.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train_dtc)
print('Training MSE: ', train_mse)
# Calculate validation error
val_mse = mean_squared_error(y_test, DTC_pred)
print('Validation MSE: ', val_mse)

Training MSE:  0.0
Validation MSE:  0.275


In [124]:
# Checking with KNN
from sklearn.neighbors import KNeighborsClassifier
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
knn = KNeighborsClassifier()

In [125]:
knn.fit(X_train,y_train)
knn_pred =knn.predict(X_test)
print(accuracy_score(y_test,knn_pred))
print(confusion_matrix(y_test,knn_pred))
print(precision_score(y_test,knn_pred))

0.675
[[ 62  56]
 [ 22 100]]
0.6410256410256411


In [126]:
# Calculate training error
y_pred_train_knn = knn.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train_knn)
print('Training MSE: ', train_mse)
# Calculate validation error
val_mse = mean_squared_error(y_test, knn_pred)
print('Validation MSE: ', val_mse)

Training MSE:  0.2833333333333333
Validation MSE:  0.325


In [127]:
# from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
rfc= RandomForestClassifier(n_estimators= 18, criterion="entropy")

In [128]:
rfc.fit(X_train,y_train)
rfc_pred =rfc.predict(X_test)
print(accuracy_score(y_test,rfc_pred))
print(confusion_matrix(y_test,rfc_pred))
print(precision_score(y_test,rfc_pred))

0.7416666666666667
[[92 26]
 [36 86]]
0.7678571428571429


In [129]:
# Calculate training error
y_pred_train_rfc = rfc.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train_rfc)
print('Training MSE: ', train_mse)
# Calculate validation error
val_mse = mean_squared_error(y_test, rfc_pred)
print('Validation MSE: ', val_mse)

Training MSE:  0.0
Validation MSE:  0.25833333333333336
